In [15]:
import requests
import os
import json
from progressIB import ProgressBar

In [17]:
cityIDs = {'Boston':"CITY:US250002",
          'NYC':"CITY:US360019",
          'Philadelphia':"CITY:US420015",
           'Newark':"CITY:US340003"}
locationid = cityIDs[city]
city = 'NYC'
if not os.path.exists(city):
    os.mkdir(city)

In [18]:
datasetid="GHCNDMS"
datatypeid=['EMXP', 'MXSD','TPCP', 
            'TSNW', 'EMXT','EMNT', 
            'MMXT', 'MMNT','MNTM']
#datatypeid='EXMP&MXSD&TPCP&TSNW&EMXT&MMXT&MMNT&MNTM'
datacategoryid=['TEMP', 'PRCP']
limit=1000
params={'datasetid':datasetid, 
        'datatypeid[]':datatypeid,
        'locationid':locationid, 
        'limit':limit}

In [19]:
all_dates = ["{}-{:02}-01".format(yr,m) for yr in range(1900,2016) for m in range(1,13) ]

In [20]:
restart_dates = all_dates[all_dates.index('1979-03-01'):]

In [21]:
p = ProgressBar(len(restart_dates))
for i, d in enumerate(restart_dates):
    params['startdate'] = d
    params['enddate'] = d
    r = requests.get("http://www.ncdc.noaa.gov/cdo-web/api/v2/data", 
                     headers={'token':token},
                    params=params)
    if 'results' not in r.json():
        print r.content
        break
                
    results = [rs for rs in r.json()['results'] if rs['datatype'] in datatypeid]
    station_data=dict()
    for rs in results:
        if rs['station'] not in station_data:
            station_data[rs['station']]=rs
        else:
            station_data[rs['station']].update(rs)
    year_data={rs['date']:station_data}
    yr, mn, _ = rs['date'].split('-')
    fname = "{}{}{}".format(city,yr,mn)
    with open(os.path.join(city,fname), 'w') as fp:
        json.dump(results, fp)
    time.sleep(0.002)
    p.animate(i)

{"status" : "429", "message" : "This token has reached its temporary request limit of 1000 per day."}
